In [ ]:
# AutoScholar Research Assistant (Kaggle)
**Capstone:** Multi-agent research assistant — search (SerpAPI) → extract → cite → summarize (Gemini)
Author: MOHAN HARI G
Date: 2025-11-20

In [ ]:
!pip install -q requests beautifulsoup4 lxml tldextract rouge-score python-dateutil tenacity

In [ ]:
import os
import json
import time
import hashlib
import logging
from pathlib import Path
from typing import List, Dict, Any
import requests
from bs4 import BeautifulSoup
import tldextract
import re
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

# Optional rouge scorer import (used in evaluation cell)
try:
    from rouge_score import rouge_scorer
except Exception:
    rouge_scorer = None

# Basic configuration
DATA_DIR = Path("data")
CACHE_DIR = DATA_DIR / "cached_results"
DATA_DIR.mkdir(exist_ok=True)
CACHE_DIR.mkdir(exist_ok=True)
MEM_FILE = DATA_DIR / "memory.json"
LOG_FILE = DATA_DIR / "logs.json"

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("autoscholar")

In [ ]:
import os
print("GEMINI key present in os.environ:", bool(os.environ.get("GEMINI_API_KEY")))

# Try to import google.generativeai and configure (do not print the key)
try:
    import google.generativeai as genai
    print("google.generativeai imported OK. genai module:", genai)
    try:
        genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
        print("Configured genai with GEMINI_API_KEY (did not print key).")
    except Exception as e:
        print("genai.configure() raised:", repr(e))
except Exception as e:
    print("Import google.generativeai failed:", repr(e))


In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    import os
    usc = UserSecretsClient()
    serp = None
    gem = None
    try:
        serp = usc.get_secret("SERPAPI_API_KEY")
    except Exception as e:
        # secret missing or not created
        serp = None
    try:
        gem = usc.get_secret("GEMINI_API_KEY")
    except Exception as e:
        gem = None

    if serp:
        os.environ["SERPAPI_API_KEY"] = serp
    if gem:
        os.environ["GEMINI_API_KEY"] = gem

    print("SERPAPI_API_KEY loaded into environment:", bool(os.environ.get("SERPAPI_API_KEY")))
    print("GEMINI_API_KEY loaded into environment:", bool(os.environ.get("GEMINI_API_KEY")))
except Exception as e:
    print("kaggle_secrets not available or permission denied:", e)
    print("If you're running outside Kaggle, add the keys to os.environ or Kaggle Secrets.")


In [ ]:
SERPAPI_API_KEY = os.environ.get("SERPAPI_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

if not SERPAPI_API_KEY:
    logger.warning("SERPAPI_API_KEY not found in environment. Add it to Kaggle Secrets before running search.")
if not GEMINI_API_KEY:
    logger.warning("GEMINI_API_KEY not found in environment. Summarizer calls will fail until it's set.")

In [ ]:
def _hash_text(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()[:16]

def save_json(path: Path, obj: Any):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, ensure_ascii=False))

def load_json(path: Path, default=None):
    if path.exists():
        return json.loads(path.read_text())
    return default

def safe_get_text(soup: BeautifulSoup) -> str:
    if soup is None:
        return ""
    # Remove unwanted tags
    for tag in soup(["script", "style", "noscript", "iframe"]):
        tag.decompose()
    text = soup.get_text(separator="\n")
    # collapse whitespace
    text = re.sub(r"\n\s*\n+", "\n\n", text)
    return text.strip()


In [ ]:
SERPAPI_SEARCH_URL = "https://serpapi.com/search"

def serpapi_search(query: str, num_results: int = 5, use_cache: bool = True) -> List[Dict[str, Any]]:
    """
    Query SerpAPI (Google engine) and return organic results as dicts.
    Caches results to data/cached_results/<hash>.json
    """
    qhash = _hash_text(f"serp::{query}::{num_results}")
    cache_path = CACHE_DIR / f"{qhash}.json"
    if use_cache and cache_path.exists():
        logger.info(f"Loading cached SerpAPI results for query: {query}")
        return load_json(cache_path, [])

    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": SERPAPI_API_KEY
    }
    try:
        resp = requests.get(SERPAPI_SEARCH_URL, params=params, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        logger.error(f"SerpAPI request failed: {e}")
        return []

    results = []
    # SerpAPI returns organic_results (may differ for location/engine)
    for item in data.get("organic_results", [])[:num_results]:
        results.append({
            "title": item.get("title"),
            "url": item.get("link") or item.get("url"),
            "snippet": item.get("snippet") or item.get("snippet_highlighted_words") or ""
        })
    # fallback: serpapi sometimes returns 'top' or 'answer_box' - ignore for now

    save_json(cache_path, results)
    logger.info(f"SerpAPI: {len(results)} results saved to cache for query: {query}")
    return results


In [ ]:
def fetch_page(url: str, timeout: int = 10) -> Dict[str, Any]:
    """Fetch a URL, return dict with title and text (fallbacks handled)."""
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (compatible; AutoScholar/1.0; +https://example.com/bot)"
        }
        r = requests.get(url, headers=headers, timeout=timeout)
        r.raise_for_status()
        html = r.text
        soup = BeautifulSoup(html, "lxml")
        title = (soup.title.string if soup.title else "") or ""
        text = safe_get_text(soup)
        return {"title": title.strip(), "text": text.strip(), "url": url}
    except Exception as e:
        logger.warning(f"Failed to fetch {url}: {e}")
        return {"title": "", "text": "", "url": url}

def extract_top_passages(page_text: str, min_chars: int = 300) -> List[str]:
    """
    Simple heuristic: split by paragraphs and return the longest passages
    useful for summarization, up to some limit.
    """
    if not page_text:
        return []
    paras = [p.strip() for p in page_text.split("\n") if len(p.strip()) > 50]
    # sort paras by length
    paras_sorted = sorted(paras, key=len, reverse=True)
    # return top 3 paragraphs that meet min_chars
    top = [p for p in paras_sorted if len(p) >= min_chars][:3]
    return top


In [ ]:
def domain_score(url: str) -> float:
    """
    Very simple domain trust heuristic:
    - educational domains (.edu) get +0.3
    - known news domains get +0.2 (simple heuristic)
    - otherwise 0
    """
    try:
        ext = tldextract.extract(url)
        suffix = ext.suffix or ""
        domain = ext.domain or ""
        score = 0.0
        if suffix.endswith("edu") or domain.endswith("edu"):
            score += 0.3
        # quick-news heuristic (you can expand)
        if domain.lower() in {"nytimes", "theguardian", "bbc"}:
            score += 0.2
        return score
    except Exception:
        return 0.0

def overlap_score(snippet: str, query: str) -> float:
    # simple token overlap ratio
    s_tokens = set(re.findall(r"\w+", snippet.lower()))
    q_tokens = set(re.findall(r"\w+", query.lower()))
    if not q_tokens:
        return 0.0
    overlap = s_tokens.intersection(q_tokens)
    return len(overlap) / max(1, len(q_tokens))

def rank_sources(results: List[Dict[str, Any]], query: str) -> List[Dict[str, Any]]:
    """
    Score sources by (overlap_score * weight) + domain_score and sort.
    Returns list with added 'score' key.
    """
    scored = []
    for r in results:
        snippet = r.get("snippet") or ""
        url = r.get("url") or ""
        base_score = overlap_score(snippet, query)
        dscore = domain_score(url)
        total = base_score * 0.7 + dscore * 0.3
        scored.append({**r, "score": total})
    scored_sorted = sorted(scored, key=lambda x: x["score"], reverse=True)
    return scored_sorted


In [ ]:
try:
    import google.generativeai as genai
    genai.configure(api_key=GEMINI_API_KEY)
    _HAS_GEMINI = True
except Exception:
    _HAS_GEMINI = False
    logger.warning("google.generativeai not available. Install library or ensure GEMINI_API_KEY is set.")

SUMMARY_PROMPT_TEMPLATE = """
You are a research assistant. Given extracted passages and their metadata, produce:
1) A factual, unbiased ~500-word summary synthesizing the main points.
2) A concise 5-bullet outline (each bullet 1–2 sentences).
3) A short list of the 3 citations (title + url + confidence).

Passages:
{passages}

Citations metadata:
{citations}

Return JSON with keys: summary, outline (list), citations (list of {title,url,note}).
"""

def call_gemini_summarizer(passages, citations, max_tokens=2048):
    """
    Calls Gemini and returns dict with {summary, outline, citations}.
    """

    if not _HAS_GEMINI:
        raise RuntimeError("Gemini client not available. Install google.generativeai and set GEMINI_API_KEY.")

    # Build the citation lines safely
    citation_lines = []
    for c in citations:
        title = c.get("title", "")
        url = c.get("url", "")
        score = c.get("score", 0)
        citation_lines.append(f"{title}: {url} (score={score:.2f})")

    prompt = SUMMARY_PROMPT_TEMPLATE.format(
        passages="\n\n---\n\n".join(passages),
        citations="\n".join(citation_lines)
    )

    # Gemini call
    response = genai.generate(
        model="gemini-1.0",
        input=prompt,
        max_output_tokens=max_tokens
    )

    # Extract text safely
    try:
        if hasattr(response, "text"):
            text = response.text
        else:
            text = str(response)
    except:
        text = str(response)

    # Try to extract JSON
    try:
        import re, json
        m = re.search(r"\{.*\}", text, re.DOTALL)
        if m:
            return json.loads(m.group(0))
    except:
        pass

    # Fallback if JSON not detected
    summary = text.strip()
    sentences = re.split(r'(?<=[.!?])\s+', summary)
    outline = sentences[:5]

    return {
        "summary": summary,
        "outline": outline,
        "citations": [
            {"title": c.get("title",""), "url": c.get("url",""), "note": f"score={c.get('score',0):.2f}"}
            for c in citations[:3]
        ]
    }


In [ ]:
class Orchestrator:
    def __init__(self, search_fn, fetch_fn, rank_fn, summarize_fn, max_search_results=6, workers=4):
        self.search_fn = search_fn
        self.fetch_fn = fetch_fn
        self.rank_fn = rank_fn
        self.summarize_fn = summarize_fn
        self.max_search_results = max_search_results
        self.workers = workers
        # memory & logs
        self.memory = load_json(MEM_FILE, {"sessions": []})
        self.logs = load_json(LOG_FILE, [])

    def _log(self, event: Dict[str, Any]):
        event["ts"] = datetime.utcnow().isoformat()
        self.logs.append(event)
        # keep log file writing cheap
        save_json(LOG_FILE, self.logs)

    def run(self, topic: str) -> Dict[str, Any]:
        run_id = _hash_text(f"run::{topic}::{time.time()}")
        self._log({"event": "start_run", "topic": topic, "run_id": run_id})
        # 1) search (parallel)
        self._log({"event": "search_start", "topic": topic})
        results = self.search_fn(topic, num_results=self.max_search_results)
        self._log({"event": "search_end", "result_count": len(results)})

        # 2) fetch + extract (parallel)
        self._log({"event": "fetch_start", "result_count": len(results)})
        pages = []
        with ThreadPoolExecutor(max_workers=self.workers) as ex:
            futures = {ex.submit(self.fetch_fn, r.get("url")): r for r in results if r.get("url")}
            for fut in as_completed(futures):
                src = futures[fut]
                try:
                    page = fut.result()
                    # attach the original snippet/title if missing
                    if not page.get("title"):
                        page["title"] = src.get("title","")
                    if not page.get("text"):
                        page["text"] = src.get("snippet","")
                    pages.append({**src, **page})
                except Exception as e:
                    logger.warning(f"Error fetching {src.get('url')}: {e}")
        self._log({"event": "fetch_end", "fetched": len(pages)})

        # 3) extract top passages to summarize
        passages = []
        for p in pages:
            top = extract_top_passages(p.get("text",""))
            for t in top:
                passages.append({"text": t, "source": {"title": p.get("title"), "url": p.get("url"), "snippet": p.get("snippet", "")}})
        # fallback: if no passages, use snippets
        if not passages:
            for r in results:
                passages.append({"text": r.get("snippet",""), "source": {"title": r.get("title"), "url": r.get("url")}})

        # 4) citation selection/ranking
        ranked = self.rank_fn(results, topic)
        top_citations = ranked[:3]

        # 5) summarizer
        # Prepare passages as plain text segments for Gemini
        passages_texts = [p["text"] for p in passages][:10]  # limit to avoid extremely long inputs
        self._log({"event": "summarizer_start", "passage_count": len(passages_texts)})
        try:
            summary_out = self.summarize_fn(passages_texts, top_citations)
        except Exception as e:
            logger.error(f"Summarizer error: {e}")
            summary_out = {"summary": "ERROR: summarizer failed.", "outline": [], "citations": [{"title": c.get("title",""), "url": c.get("url","")} for c in top_citations]}

        self._log({"event": "summarizer_end", "summary_len": len(summary_out.get("summary","")) if summary_out.get("summary") else 0})

        # 6) persist to memory
        session_record = {
            "run_id": run_id,
            "topic": topic,
            "timestamp": datetime.utcnow().isoformat(),
            "summary": summary_out.get("summary",""),
            "outline": summary_out.get("outline", []),
            "citations": summary_out.get("citations", [])
        }
        self.memory.setdefault("sessions", []).append(session_record)
        save_json(MEM_FILE, self.memory)
        self._log({"event": "run_end", "run_id": run_id})

        # final structured output
        return {
            "topic": topic,
            "summary": summary_out.get("summary",""),
            "outline": summary_out.get("outline", []),
            "citations": summary_out.get("citations", []),
            "meta": {"results_found": len(results), "fetched_pages": len(pages)}
        }


In [ ]:
orchestrator = Orchestrator(
    search_fn=serpapi_search,
    fetch_fn=fetch_page,
    rank_fn=rank_sources,
    summarize_fn=call_gemini_summarizer,
    max_search_results=6,
    workers=4
)
logger.info("Orchestrator created. Ready to run.")


In [ ]:

import google.generativeai as genai
import json, re, traceback

genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))

def call_gemini_summarizer(passages, citations, max_tokens=2048):
    # Normalize citations
    safe_citations = []
    for c in citations or []:
        if isinstance(c, dict):
            safe_citations.append({
                "title": c.get("title",""),
                "url": c.get("url",""),
                "score": float(c.get("score",0) or 0)
            })
    if not safe_citations:
        safe_citations = [{"title":"No source found","url":"","score":0.0}]
    
    citation_lines = [
        f"{c['title']}: {c['url']} (score={c['score']:.2f})"
        for c in safe_citations
    ]
    
    prompt = SUMMARY_PROMPT_TEMPLATE.format(
        passages="\n\n---\n\n".join(passages or [""]),
        citations="\n".join(citation_lines)
    )
    
    try:
        # Use correct Gemini API for your environment
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)

        # Text output
        text = response.text

        # Try extract JSON
        m = re.search(r"\{.*\}", text, re.DOTALL)
        if m:
            parsed = json.loads(m.group(0))
            parsed.setdefault("summary","")
            parsed.setdefault("outline", [])
            parsed.setdefault("citations", [])

            return parsed

        # Fallback: simple text summary
        summary = text.strip()
        sentences = re.split(r'(?<=[.!?])\s+', summary)
        outline = sentences[:5]

        return {
            "summary": summary,
            "outline": outline,
            "citations": [
                {"title": c["title"], "url": c["url"], "note": f"score={c['score']:.2f}"}
                for c in safe_citations[:3]
            ]
        }

    except Exception as e:
        print("Gemini error:", e)
        # Extractive fallback
        joined = "\n\n".join(passages or [])
        summary = joined[:1200] or "No content to summarize."
        sents = re.split(r'(?<=[.!?])\s+', summary)
        outline = sents[:5]
        return {
            "summary": summary,
            "outline": outline,
            "citations": [
                {"title": c["title"], "url": c["url"], "note": f"score={c['score']:.2f}"}
                for c in safe_citations[:3]
            ]
        }

print("Replaced summarizer with GenerativeModel-based version.")


In [ ]:
out = orchestrator.run("Impact of Generative AI on developer productivity")
print(out["summary"][:1000])

In [ ]:

from pprint import pprint
logs = load_json(LOG_FILE, []) or []
print("Total log events:", len(logs))
gem_errs = [e for e in logs if e.get("event") in ("gemini_error","gemini_exception","gemini_call_failed")]
print("gemini_error-like events:", len(gem_errs))
for e in gem_errs[-10:]:
    print("="*80)
    print("ts:", e.get("ts"))
    print("event:", e.get("event"))
    print("error:", e.get("error"))
    trace = e.get("trace") or e.get("traceback") or e.get("tb") or ""
    if trace:
        print("trace (trimmed):")
        print(trace[:2000])
    else:
        # sometimes error stored as plain message
        print("raw:", e)
print("="*80)
print("\nLast 20 generic log events (most recent last):")
for ev in logs[-20:]:
    print(ev.get("ts","")[:19], ev.get("event"), ev.get("topic",""), ev.get("summary_len", ev.get("result_count","")))


In [ ]:

import traceback, inspect
try:
    import google.generativeai as genai
    genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
    print("Using google.generativeai version:", getattr(genai, "__version__", "unknown"))
    # Try a short prompt and capture full response object
    model_name_candidates = ["gemini-pro", "gemini-2.5-flash" ,"gemini-1.0", "gemini-1.5", "gemini-1.5-pro", "gemini-1.0-lite", "chat-bison"]
    tried = []
    resp = None
    # try the model you used earlier first
    for model_name in [ "gemini-2.5-flash", "gemini-1.0" ] + model_name_candidates:
        try:
            print("\nTrying model:", model_name)
            model = genai.GenerativeModel(model_name)
            # Use a very short prompt to reduce cost and time
            resp = model.generate_content("Say hello in one sentence.", max_output_tokens=32)
            print("Call succeeded with model:", model_name)
            break
        except Exception as e:
            print(" -> call failed for", model_name, ":", repr(e))
            tried.append((model_name, repr(e)))
            continue

    if resp is None:
        print("All model attempts failed. Summary of attempts:")
        for t in tried:
            print(t[0], ":", t[1])
    else:
        print("\n=== RAW RESPONSE REPR ===")
        print(repr(resp)[:2000])
        print("\n=== TYPE ===")
        print(type(resp))
        print("\n=== dir(resp) sample ===")
        attr_names = [a for a in dir(resp) if not a.startswith("_")]
        print(attr_names[:60])
        # print any obvious text attributes
        for key in ("text","content","result","candidates","candidates_text","content_text"):
            if hasattr(resp, key):
                try:
                    print(f"\nATTRIBUTE resp.{key} (preview):")
                    val = getattr(resp, key)
                    print(repr(val)[:2000])
                except Exception as e:
                    print("error reading attribute", key, e)
        # try to iterate any top-level fields
        try:
            # some response objects behave like mappings
            items = list(resp.__dict__.items())[:20]
            print("\nresp.__dict__ sample (trimmed):")
            for k,v in items:
                print(k, ":", repr(v)[:400])
        except Exception:
            pass
except Exception:
    print("Exception in diagnostic:")
    traceback.print_exc()


In [ ]:

def extractive_only_summarizer(passages, citations, max_tokens=1024):
    # create a ~500-word extractive summary from passages
    joined = "\n\n".join(passages or [])
    if not joined.strip():
        return {"summary":"No passages available to summarize.","outline":[],"citations":[{"title":c.get("title",""),"url":c.get("url",""),"note":c.get("note","")} for c in (citations or [])[:3]]}
    paras = [p.strip() for p in joined.split("\n\n") if p.strip()]
    paras_sorted = sorted(paras, key=len, reverse=True)
    word_target = 500
    summary_parts = []
    current_words = 0
    for p in paras_sorted:
        wc = len(p.split())
        if current_words >= word_target:
            break
        summary_parts.append(p)
        current_words += wc
    # if still not enough, fill with remaining text truncated
    if current_words < word_target:
        # just take joined truncated
        summary_text = (joined[:4000]).strip()
    else:
        summary_text = "\n\n".join(summary_parts)
    # outline: pick 5 longest sentences
    sents = re.split(r'(?<=[.!?])\s+', summary_text)
    sents_sorted = sorted([s for s in sents if s.strip()], key=len, reverse=True)
    outline = [s.strip() for s in sents_sorted[:5]]
    # normalize citations
    norm_cits = []
    for c in (citations or [])[:3]:
        if isinstance(c, dict):
            norm_cits.append({"title": c.get("title",""), "url": c.get("url",""), "note": c.get("note", f"score={c.get('score','')}")})
        else:
            norm_cits.append({"title": str(c), "url": "", "note": ""})
    return {"summary": summary_text, "outline": outline, "citations": norm_cits}


call_gemini_summarizer = extractive_only_summarizer
orchestrator.summarize_fn = call_gemini_summarizer  if hasattr(orchestrator, "summarize_fn") else None
print("Now using extractive-only summarizer. Re-run a demo topic to verify output.")


In [ ]:

demo_topics = [
    "Impact of Generative AI on developer productivity",
    "Edge computing vs cloud computing for smart city applications",
    "Use of LLMs in healthcare diagnostics — opportunities and risks",
    "Climate change: projected sea-level rise in South and Southeast Asia",
    "Cybersecurity best practices for small businesses"
]


for topic in demo_topics:
    print("\n" + "="*80)
    print("TOPIC:", topic)
    out = orchestrator.run(topic)
    print("\n--- SUMMARY (first 800 chars) ---\n")
    print(out["summary"][:800] + ("\n..." if len(out["summary"])>800 else "\n"))
    print("\n--- OUTLINE ---")
    for b in out["outline"][:5]:
        print("-", b)
    print("\n--- CITATIONS ---")
    for c in out["citations"][:3]:
        if isinstance(c, dict):
            print(f"- {c.get('title','')} — {c.get('url','')} (note: {c.get('note', '') or c.get('score', '')})")
        else:
            print("-", c)
    print("\nMeta:", out["meta"])


In [ ]:

import pandas as pd

def export_human_eval_csv(topics_outputs: List[Dict[str, Any]], out_path: str = "data/human_eval_template.csv"):
    rows = []
    for o in topics_outputs:
        rows.append({
            "topic": o["topic"],
            "summary": o["summary"],
            "outline": " | ".join(o["outline"]),
            "citation1": o["citations"][0]["url"] if o["citations"] and len(o["citations"])>0 else "",
            "citation2": o["citations"][1]["url"] if o["citations"] and len(o["citations"])>1 else "",
            "citation3": o["citations"][2]["url"] if o["citations"] and len(o["citations"])>2 else "",
            "relevance": "", "coherence": "", "citations_correctness": "", "notes": ""
        })
    df = pd.DataFrame(rows)
    df.to_csv(out_path, index=False)
    return df

# Save human-eval CSV for all demo topics (reads from orchestrator.memory)
outputs = orchestrator.memory.get("sessions", [])[-len(demo_topics):] if orchestrator.memory.get("sessions") else []
df = export_human_eval_csv(outputs, out_path="data/human_eval_template.csv")
print("Human-eval CSV saved to data/human_eval_template.csv")
df.head()


In [ ]:

if rouge_scorer is None:
    print("rouge_score library not available. Install `rouge-score` to compute automatic ROUGE metrics.")
else:
    # Example: If you have a dict `references` mapping topic->reference_summary
    references = {}  # fill in if you have references
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    rows = []
    for s in orchestrator.memory.get("sessions", []):
        topic = s["topic"]
        pred = s["summary"]
        ref = references.get(topic)
        if not ref:
            continue
        scores = scorer.score(ref, pred)
        rows.append({"topic": topic, **{k: v.fmeasure for k,v in scores.items()}})
    if rows:
        import pandas as pd
        df = pd.DataFrame(rows)
        print(df.describe().T)
    else:
        print("No reference summaries provided; skipping ROUGE.")


In [ ]:
print("Recent sessions saved:", len(orchestrator.memory.get("sessions", [])))
# show last 2 sessions
for s in orchestrator.memory.get("sessions", [])[-2:]:
    print("->", s["topic"], "|", s["timestamp"], "| summary_len:", len(s.get("summary","")))
    
# Show logs (limited)
logs = load_json(LOG_FILE, [])
print(f"Loaded {len(logs)} log events. Last 10:")
for ev in logs[-10:]:
    print(ev.get("ts"), ev.get("event"), ev.get("topic", ""), ev.get("summary_len", ev.get("result_count", "")))


In [ ]:
save_json(MEM_FILE, orchestrator.memory)
save_json(LOG_FILE, orchestrator.logs)
print("Memory and logs saved to data/")


In [ ]:
from pprint import pprint
import pandas as pd

# 1) define the single topic (first of the demo list)
topic = "Impact of Generative AI on developer productivity"

print(f"Running AutoScholar on single topic:\n-> {topic}\n")
out = orchestrator.run(topic)

# 2) Pretty-print results (summary preview + outline + citations)
print("\n" + "="*80)
print("SUMMARY (first 1200 chars):\n")
print(out["summary"][:1200] + ("\n..." if len(out["summary"])>1200 else "\n"))
print("\n" + "-"*40)
print("OUTLINE:")
for i, b in enumerate(out.get("outline", [])[:5], 1):
    print(f"{i}. {b}")

print("\n" + "-"*40)
print("CITATIONS:")
for i, c in enumerate(out.get("citations", [])[:3], 1):
    title = c.get("title","")
    url = c.get("url","")
    note = c.get("note","") or c.get("score","")
    print(f"{i}. {title}\n   {url}\n   note: {note}")

print("\nMeta:", out.get("meta"))
print("="*80 + "\n")

# 3) Export human-eval CSV (single row)
row = {
    "topic": out["topic"],
    "summary": out["summary"],
    "outline": " | ".join(out.get("outline", [])),
    "citation1": out.get("citations", [])[0].get("url","") if out.get("citations") else "",
    "citation2": out.get("citations", [])[1].get("url","") if len(out.get("citations", []))>1 else "",
    "citation3": out.get("citations", [])[2].get("url","") if len(out.get("citations", []))>2 else "",
    "relevance": "", "coherence": "", "citations_correctness": "", "notes": ""
}
df = pd.DataFrame([row])
df.to_csv("data/human_eval_single_topic.csv", index=False)
print("Human-eval CSV saved to: data/human_eval_single_topic.csv")
display(df.head(1))


In [ ]:

import pandas as pd

logs = load_json(LOG_FILE, []) or []
df_logs = pd.DataFrame(logs[-50:])
print("Showing last 50 log events:")
df_logs


In [ ]:

import matplotlib.pyplot as plt

if not logs:
    print("No logs to display.")
else:
    times = [l.get("ts") for l in logs]
    events = [l.get("event") for l in logs]

    plt.figure(figsize=(14,4))
    plt.plot(range(len(times)), range(len(times)), alpha=0)  # dummy axis

    for i, (t, e) in enumerate(zip(times, events)):
        plt.scatter(i, 0, s=40)
        plt.text(i, 0.03, e, rotation=45, fontsize=8, ha='right')

    plt.title("Timeline of Agent Events")
    plt.yticks([])
    plt.xlabel("Event Index (chronological)")
    plt.show()


In [ ]:

from collections import Counter

event_counts = Counter([l.get("event") for l in logs])
print("Tool / Event usage counts:\n")
for k, v in event_counts.items():
    print(f"{k:20s} : {v}")


In [ ]:

import pandas as pd
from datetime import datetime

def parse_ts(ts):
    try:
        return datetime.fromisoformat(ts)
    except:
        return None

df = pd.DataFrame(logs)
df["dt"] = df["ts"].apply(parse_ts)

latencies = {}

# stage: search
start = df[df.event == "search_start"]["dt"].max()
end   = df[df.event == "search_end"]["dt"].max()
if start and end:
    latencies["search_ms"] = (end - start).total_seconds() * 1000

# stage: fetch
start = df[df.event == "fetch_start"]["dt"].max()
end   = df[df.event == "fetch_end"]["dt"].max()
if start and end:
    latencies["fetch_ms"] = (end - start).total_seconds() * 1000

# stage: summarizer
start = df[df.event == "summarizer_start"]["dt"].max()
end   = df[df.event == "run_end"]["dt"].max()
if start and end:
    latencies["summarizer_ms"] = (end - start).total_seconds() * 1000

latencies

In [ ]:
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

In [ ]:
!adk create AutoScholar_Research_Assistant_agent --model gemini-2.5-flash-lite --api_key $GEMINI_API_KEY

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini

from google.genai import types

# Configure Model Retry on errors
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def set_device_status(location: str, device_id: str, status: str) -> dict:
    """Sets the status of a smart home device.

    Args:
        location: The room where the device is located.
        device_id: The unique identifier for the device.
        status: The desired status, either 'ON' or 'OFF'.

    Returns:
        A dictionary confirming the action.
    """
    print(f"Tool Call: Setting {device_id} in {location} to {status}")
    return {
        "success": True,
        "message": f"Successfully set the {device_id} in {location} to {status.lower()}."
    }

# This agent has DELIBERATE FLAWS that we'll discover through evaluation!
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoScholar_Research_Assistant_agent",
    description="A multi-agent research system that searches, extracts, summarizes, and cites reliable information—automating end-to-end topic analysis.",
    instruction="""AutoScholar works by taking any topic you give it and starting a full research workflow automatically. It searches the web, collects useful information, extracts key points, and generates a clear summary with citations. You only need to run the notebook cell with your topic, and the agent handles everything else on its own. When it finishes, you can read the summary, check the outline, see the sources, and view the logs of how the agent worked.""",
    tools=[set_device_status],
)

In [ ]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}

In [ ]:
rm -rf AUTOSCHOLAR_automation_agent


In [ ]:
rm -rf home_automation_agent